<a href="https://colab.research.google.com/github/lightballpikachu/SummerAnalytics/blob/main/CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model 1:

In [ ]:
!pip install bokeh

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import bokeh.plotting
import panel as pn

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

df = pd.read_csv("/content/dataset.csv")
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")
df = df[["Timestamp", "Occupancy", "Capacity"]]
df["Day"] = df["Timestamp"].dt.date

daily_summary = df.groupby("Day").agg({
    "Occupancy": ["max", "min"],
    "Capacity": "max"
}).reset_index()

daily_summary.columns = ["Day", "OccMax", "OccMin", "Capacity"]
daily_summary["Price"] = 10 + (daily_summary["OccMax"] - daily_summary["OccMin"]) / daily_summary["Capacity"]

source = ColumnDataSource(data={
    "Day": pd.to_datetime(daily_summary["Day"]),
    "Price": daily_summary["Price"]
})

fig = figure(x_axis_type="datetime", title="Model 1: Daily Parking Price", width=800, height=400)
fig.line(x="Day", y="Price", source=source, line_width=2, color="navy")
fig.circle(x="Day", y="Price", source=source, size=6, color="red")

show(fig)


We begin with a simple baseline model where the price increases linearly with the occupancy rate of the parking lot.
The formula used is:

Price=10+𝛼⋅(Occupancy/Capacity)

Here, we chose 𝛼=5 to scale prices moderately. This allows for variation from $10 to $15+ depending on how full the lot is. Prices are bounded between $5 and $20 to keep them realistic.

This model serves as a reference to evaluate more intelligent models later.

Model 2:

In [ ]:
!pip install bokeh

import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

# Load dataset
df = pd.read_csv("/content/dataset.csv")
df = df[df["Capacity"] > 0]

# Timestamp construction
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")
df["Timestamp"] = df["Timestamp"].dt.round("30min")

# Numeric conversions
df["Capacity"] = pd.to_numeric(df["Capacity"], errors="coerce")
df["Occupancy"] = pd.to_numeric(df["Occupancy"], errors="coerce")
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

# Vehicle weight mapping
vehicle_weights = {"car": 1.0, "bike": 0.6, "truck": 1.5}
df["VehicleTypeWeight"] = df["VehicleType"].str.lower().map(vehicle_weights).fillna(1.0)

# Convert feature columns to numeric
df["QueueLength"] = pd.to_numeric(df["QueueLength"], errors="coerce")
df["TrafficConditionNearby"] = pd.to_numeric(df["TrafficConditionNearby"], errors="coerce")
df["IsSpecialDay"] = pd.to_numeric(df["IsSpecialDay"], errors="coerce")

# Demand formula
df["Demand"] = (
    2.0 * df["OccupancyRate"] +
    1.0 * df["QueueLength"] +
    0.3 * df["IsSpecialDay"] +
    0.8 * df["VehicleTypeWeight"]
)


# Normalize demand
df["Demand"] = df["Demand"].fillna(0)
d_min, d_max = df["Demand"].min(), df["Demand"].max()
df["NormDemand"] = (df["Demand"] - d_min) / (d_max - d_min + 1e-6)

# Final price
df["RawPrice"] = 10 * (1 + 0.6 * df["NormDemand"])
df["Price"] = df["RawPrice"].clip(lower=5, upper=20)

# Prepare for plotting
df = df.dropna(subset=["Price", "Timestamp"])
df = df.sort_values("Timestamp").reset_index(drop=True)

source = ColumnDataSource(data={
    "Timestamp": df["Timestamp"],
    "Price": df["Price"]
})

# Bokeh plot
fig = figure(x_axis_type="datetime", title="Model 2: Demand-Based Pricing", width=800, height=400)
fig.line(x="Timestamp", y="Price", source=source, line_width=2, color="navy")
fig.circle(x="Timestamp", y="Price", source=source, size=5, color="orange")

show(fig)


Model 2 improves on the baseline by introducing a demand function that includes multiple real-world features. The price adjusts based on:

Occupancy rate (how full the lot is)

Queue length (indicates excess demand)

Special day indicator (e.g., holidays)

Vehicle type (heavier/larger vehicles take more space)

The demand function:

Demand=2.0⋅OccupancyRate+1.0⋅QueueLength+0.3⋅IsSpecialDay+0.8⋅VehicleTypeWeight

We normalize demand and use it to adjust the price:

Price=10⋅(1+0.6⋅NormalizedDemand)

Prices are bounded between $5 and $20. This model captures more behavioral context and produces realistic price variation across lots and time.